In [1]:
'''script to regrid CMIP6 datatsets to target grid and store them'''

import numpy as np
import xarray as xr
import dask
import intake
import pandas as pd
import os
from collections import defaultdict
from tqdm.autonotebook import tqdm
from xmip.utils import google_cmip_col
from xmip.postprocessing import combine_datasets, _match_datasets,_concat_sorted_time,concat_members
from cmip_catalogue_operations import reduce_cat_to_max_num_realizations, drop_vars_from_cat, drop_older_versions
from cmip_ds_dict_operations import select_period, pr_flux_to_m, drop_duplicate_timesteps, drop_coords, drop_incomplete
import xesmf as xe
import gcsfs
fs = gcsfs.GCSFileSystem() #list stores, stripp zarr from filename, load 

/tmp/ipykernel_8780/933853136.py:10: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
def get_lonlat_idx_nearest_to_tgs(tg_ds,ds):
    '''tg_ds = xr.DataSet containing 'lon' and 'lat' coordinates of tide gauges
    ds    = xr.DataSet containing CMIP6 data to subset
    '''
    lon_name = list(k for k in ds.dims if 'lon' in k)[0] #find lon/lat coordinate names
    lat_name = list(k for k in ds.dims if 'lat' in k)[0]
    
    #compute distances between TG coordinates and grid cell centers
    distances = 2*np.arcsin( np.sqrt(
        np.sin( (np.pi/180) * 0.5*(ds[lat_name]-tg_ds.lat) )**2 +
        np.cos((np.pi/180)*tg_ds.lat)*np.cos((np.pi/180)*ds[lat_name])*np.sin((np.pi/180)*0.5*(ds[lon_name]-tg_ds.lon))**2) )
    
    idx_nearest = distances.argmin(dim=[lon_name,lat_name]) #find indices of nearest grid cells
    return idx_nearest

In [3]:
#configure settings
output_path = 'gs://leap-persistent/timh37/CMIP6/timeseries_eu_gesla2_tgs/test'
overwrite_existing = True #whether or not to process files for which output already exists (to-do: implement)

tg_coords = xr.open_dataset('/home/jovyan/CMIP6cex/cmip6_processing/gssr_mlr_coefs_1p5_9deg_gesla2.nc')

query_vars = ['sfcWind','pr'] #variables to process
required_vars = ['sfcWind','pr','psl'] #variables that includes models should provide

ssps = ['ssp245','ssp585']

In [4]:
#query simulations & manipulate data catalogue:
col = google_cmip_col() #google cloud catalogue
qc_col = intake.open_esm_datastore("https://storage.googleapis.com/leap-persistent-ro/data-library/catalogs/cmip6-test/leap-pangeo-cmip6-test.json") #temporary pangeo-leap-forge catalogue
noqc_col = intake.open_esm_datastore("https://storage.googleapis.com/leap-persistent-ro/data-library/catalogs/cmip6-test/leap-pangeo-cmip6-noqc-test.json")

col_df = col.df
qc_df = qc_col.df
nonqc_df = noqc_col.df

col_df['prio'] = 2
qc_df['prio'] = 3
nonqc_df['prio'] = 1

col.esmcat._df = pd.concat([col_df,qc_df,nonqc_df],ignore_index=True) #merge these catalogues
ssp_cats = defaultdict(dict)

#search catalogue per ssp (need to do this for each SSP separately as availability may differ between them)
for s,ssp in enumerate(ssps):
    ssp_cat = col.search( #find instances providing all required query_vars for both historical & ssp experiments
    experiment_id=['historical',ssp],
    table_id='day',
    variable_id=required_vars,
    require_all_on=['source_id', 'member_id','grid_label'])
    ssp_cats[ssp] = ssp_cat
    
ssp_cats_merged = ssp_cats[ssp] #merge catalogues for all ssps, and drop duplicate historical simulations
ssp_cats_merged.esmcat._df = pd.concat([v.df for k,v in ssp_cats.items()],ignore_index=True).drop_duplicates(ignore_index=True)
ssp_cats_merged = drop_older_versions(ssp_cats_merged) #if google cloud and leap-pangeo catalogues provide duplicate datasets, keep the newest version, and if the versions are identical, keep the leap-pangeo dataset
ssp_cats_merged = reduce_cat_to_max_num_realizations(ssp_cats_merged) #per model, select grid and 'ipf' combination providing most realizations (needs to be applied to both SSPs together to ensure the same variants are used under both scenarios)

In [ ]:
for s,ssp in tqdm(enumerate(['ssp245'])): #for each ssp:  
    #select historical and ssp data in merged catalogue for this particular ssp
    cat_to_open = ssp_cats_merged.search(
    experiment_id=['historical',ssp],
    table_id='day',
    variable_id=required_vars,
    require_all_on=['source_id', 'member_id','grid_label'])

    cat_to_open = drop_vars_from_cat(cat_to_open,[k for k in required_vars if k not in query_vars]) #out of required variables only process query variables
    #open datasets into dictionary
    cat_to_open.esmcat.aggregation_control.groupby_attrs = [] #to circumvent aggregate=false bug

    #to avoid this issue: https://github.com/intake/intake-esm/issues/496
        #doesn't actually aggregate if we set cmip6_cat.esmcat.aggregation_control.groupby_attrs = []
    kwargs = {'zarr_kwargs':{'consolidated':True,'use_cftime':True},'aggregate':True} #keyword arguments for generating dictionary of datasets from cmip6 catalogue
    ddict = cat_to_open.to_dataset_dict(**kwargs) #open datasets into dictionary

    #preprocess datasets in dictionary
    ddict = pr_flux_to_m(ddict) #convert pr flux to accumulated pr
    ddict = drop_duplicate_timesteps(ddict) #remove duplicate timesteps if datasets have them
    #ddict = select_period(ddict,1850,2100) #preselect time periods, do this at later stage in the chain?
    ddict = drop_coords(ddict,['bnds','nbnd','height']) #remove some unused auxiliary coordinates
    
    #concatenate historical and ssp datasets in time
    with dask.config.set(**{'array.slicing.split_large_chunks': True}):
        hist_ssp = combine_datasets(ddict,_concat_sorted_time,match_attrs =['source_id', 'grid_label','table_id','variant_label','variable_id'],combine_func_kwargs={'join':'inner','coords':'minimal'})    

    hist_ssp_ = defaultdict(dict) #probably a better way to do this, but there are approx. 1 files for which the time units are inconsistent between historical and ssp
    for k,v in hist_ssp.items():
        if v.time[-1].values.dtype != v.time[0].values.dtype:
            print('dropping ' + k +' due to inconsistent timestamps in historical and ssp runs')
            continue
        else:
            hist_ssp_[k] = v
            
    hist_ssp_ = drop_duplicate_timesteps(hist_ssp_) #remove overlap between historical and ssp experiments which sometimes exists
    #hist_ssp_complete = drop_incomplete(hist_ssp_) #remove historical+ssp timeseries which are not montonically increasing or have large timegaps (based on Julius Buseckes rudimentary testing in CMIP6-LEAP-feadstock)
    hist_ssp_complete= hist_ssp_
    
    #regrid these datasets to the target tide gauges
    ddict_at_tgs = defaultdict(dict)
    for key,ds in tqdm(hist_ssp_complete.items()):
        variable = key.split('.')[-1]
        #check if dataset was already stored
        output_fn = os.path.join(output_path,variable,ds.source_id,key+'.hist_'+ssp) #output filename
        try:
            if (~overwrite_existing) & (output_fn.replace('gs://','') in fs.ls(os.path.join(output_path,variable,ds.source_id))):
                continue
        except:
            pass
            
        ds.attrs["time_concat_key"] = key #add current key information to attributes
        ds = ds.isel(dcpp_init_year=0,drop=True) #remove this coordinate
        
        if ('missing_value' in ds[variable].encoding) & ('_FillValue' in ds[variable].encoding):
            if ds[variable].encoding['missing_value'] != ds[variable].encoding['_FillValue']:
                del ds[variable].encoding['missing_value']
                
        idx_nearest = get_lonlat_idx_nearest_to_tgs(tg_coords,ds)
        lon_name = list(k for k in ds.dims if 'lon' in k)[0] #find lon/lat coordinate names
        lat_name = list(k for k in ds.dims if 'lat' in k)[0]

        ds_at_tgs = ds.isel({lat_name:idx_nearest[lat_name],lon_name:idx_nearest[lon_name]}) #this leads the kernel to crash, but simply just indexing the first 100 latitudes is fine?
        ds_at_tgs = ds_at_tgs.chunk({'time':len(ds_at_tgs.time),'tg':len(ds_at_tgs.tg)})
        
        ds_at_tgs['tg'] = ds_at_tgs.tg.astype('str') #something wrong with encoding object types in zarr, this is the work-around
        
        ddict_at_tgs[key] = ds_at_tgs
        ds.to_zarr(output_fn,mode='w')

0it [00:00, ?it/s]


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.zstore.dcpp_init_year.version.prio'


Dropping duplicate timesteps for:CESM2-WACCM.gn.day.r1i1p1f1.pr
Dropping duplicate timesteps for:FGOALS-g3.gn.day.r1i1p1f1.pr
Dropping duplicate timesteps for:CESM2-WACCM.gn.day.r3i1p1f1.pr
Dropping duplicate timesteps for:FGOALS-g3.gn.day.r3i1p1f1.pr
Dropping duplicate timesteps for:CESM2-WACCM.gn.day.r2i1p1f1.pr
Dropping duplicate timesteps for:CESM2-WACCM.gn.day.r3i1p1f1.sfcWind
Dropping duplicate timesteps for:CESM2.gn.day.r4i1p1f1.pr
Dropping duplicate timesteps for:CESM2.gn.day.r11i1p1f1.sfcWind
Dropping duplicate timesteps for:CESM2-WACCM.gn.day.r1i1p1f1.sfcWind
Dropping duplicate timesteps for:CESM2.gn.day.r4i1p1f1.sfcWind
Dropping duplicate timesteps for:CESM2.gn.day.r11i1p1f1.pr
Dropping duplicate timesteps for:CESM2-WACCM.gn.day.r2i1p1f1.sfcWind
Dropping duplicate timesteps for:EC-Earth3-Veg.gr.day.r5i1p1f1.pr
Dropping duplicate timesteps for:EC-Earth3-Veg.gr.day.r5i1p1f1.sfcWind


  0%|          | 0/532 [00:00<?, ?it/s]